# Logistic Regression
### Definisi Logistic Regression
Logistic Regression adalah model regresi yang menggunakan fungsi logistic. Model ini digunakan untuk memprediksi probabilitas dari kelas tertentu. Setiap probabilitas akan bernilai antara 0 dan 1 dengan total probabilitas yang bernilai 1.

### Mengimpor Library yang Digunakan


In [ ]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

### Memuat Dataset
Pada percobaan kali ini menggunakan dataset kanker payudara yang bersumber dari [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Original%29) dan dapat diimpor melalui library sklearn.

In [ ]:
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target
n_samples, n_features = X.shape
print(f"Banyaknya sampel data: {n_samples} dan banyaknya fitur: {n_features}")

### Mencoba mencetak tabel dengan pandas
Dengan menggunakan library pandas, kita dapat mencetak tabel yang berisi data dan target. Sebagai contoh, akan dicetak sebagian data, dengan 30 fitur (kolom) dan 10 sampel (baris)

In [ ]:

import pandas as pd
df = pd.DataFrame(X, columns=bc.feature_names)
df['target'] = y
df.head(10)

### Training dan Testing Split
Percobaan ini akan menggunakan 80% data dari dataset untuk keperluan training dan 20% data untuk keperluan testing. Untuk memisahkan data, digunakan fungsi ```train_test_split``` yang dapat diimpor dari library ```sklearn.model_selection```.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Banyaknya sampel untuk training: {X_train.shape[0]} dan banyaknya sampel untuk testing: {X_test.shape[0]}")

### Mencoba untuk memplot data training dan testing menggunakan fitur pertama dan kedua

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap='viridis', s=60, edgecolor='k')
plt.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap='viridis', s=60, edgecolor='k')
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")
plt.title("Data Training dan Data Testing")
plt.show()

### Melakukan Transformasi Data

In [ ]:
# membuat scaler untuk data training
scaler = StandardScaler()
# mentransformasikan data training dan testing
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#plot data training dan testing
plt.figure(figsize=(10, 6))
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap='viridis', s=60, edgecolor='k')
plt.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap='viridis', s=60, edgecolor='k')
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")
plt.title("Data Training dan Data Testing Setelah di Transformasi")
plt.show()

Dapat kita amati bahwa hasil plot data training dan testing setelah dilakukan scaler akan berubah. Proses ini disebut standarisasi data (data standardization), yaitu proses normalisasi data sehingga data memiliki mean=0 dan variance=1.

- ```fit_transform``` digunakan untuk training data. Metode fit mengkalkulasi mean dan variance dari setiap fitur data. Transformasi ini mentransformasikan setiap fitur menggunakan mean dan variance yang telah dikalkulasi.
- ```transform``` digunakan untuk data testing. Perbedaannya, transform menggunakan nilai mean dan variance yang telah dikalkulasi dari training data untuk ditransformasikan terhadap data testing.

**Mengapa kita perlu melakukan ini?**
Jika kita menggunakan metode ```fit_transform``` pada data testing, kita akan menghitung ulang mean dan varians. Ini adalah prosedur standar untuk menskalakan data pada saat membangun model pembelajaran mesin.


### Persiapan Data Sebelum Training

In [ ]:
# mengkonversi data training dan testing ke tensor
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

# melakukan reshape terhadap y_test dan y_train
y_train = y_train.view(-1, 1)
y_test = y_test.view(-1, 1)

### Membangun Model
Model yang digunakan adalah logistic regression dengan menggunakan library ```torch.nn```. Sementara untuk fungsi aktivasi, kita menggunakan fungsi ```Sigmoid```. Fungsi sigmoid akan menghasilkan nilai keluaran 0 atau 1 berdasarkan dari nilai yang dihasilkan oleh fungsi.

In [ ]:
class LogisticRegression(nn.Module):
    def __init__(self, input_size, output_size=1):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, output_size)

    def forward(self, x):
        y_predicted = torch.sigmoid(self.linear(x))
        return y_predicted


model = LogisticRegression(n_features)

### Fungsi Loss dan Optimizer
Fungsi loss yang digunakan adalah [Binary Cross Enthropy](https://pytorch.org/docs/stable/generated/torch.nn.BCELoss.html). Sementara itu, optimizer yang digunakan adalah [Stochastic Gradient Descent](https://pytorch.org/docs/stable/optim.html#torch.optim.SGD).

In [ ]:
learning_rate = 0.01
loss_function = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

### Training

In [ ]:
# training loop
num_epoch = 100
for epoch in range(num_epoch):
    # forward pass
    y_predicted = model(X_train)
    # backward pass
    loss = loss_function(y_predicted, y_train)
    loss.backward()
    # perbarui weight
    optimizer.step()
    # reset optimizer jadi 0
    optimizer.zero_grad()

    # menampilkan loss setiap epoch
    if (epoch+1) % 10 == 0:
        print(f"Epoch: {epoch+1}, Loss: {loss.item():.4f}")

### Mengevaluasi performa

In [ ]:
with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum().float() / y_test.shape[0]
    print(f"Akurasi: {acc.item():.4f}")

# perbandingkan antara y_test dan y_predicted_cls menggunakan pandas
df_test = pd.DataFrame(y_test.numpy(), columns=['y_test'])
df_test['y_predicted_cls'] = y_predicted_cls.numpy()
df_test['Kesesuaian'] = df_test['y_test'] == df_test['y_predicted_cls']
df_test

In [ ]:
# cetak jumlah salah dan benar
print(f"Jumlah salah: {(df_test['Kesesuaian'] == False).sum()}")
print(f"Jumlah benar: {(df_test['Kesesuaian'] == True).sum()}")